# Figure 9

## Imports

In [ ]:
import dolfin  
import matplotlib.pyplot as plt
import numpy
import sympy
import sys

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

from matplotlib.collections import PatchCollection

## Defining Geometry

In [ ]:
seeds_filename = "Fig9-seeds.dat"
mesh_filebasename = "Fig9-mesh"

domain_y = 1
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vol = (xmax - xmin)*(ymax - ymin)
dV = dolfin.Measure("dx",domain=mesh)
Phi_s0 = dolfin.assemble(1*dV)/vol

vertices = numpy.array([[xmin, ymin],
                     [xmax, ymin],
                     [xmax, ymax],
                     [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

## Defining kinematics

In [ ]:
alpha_x, alpha_y, alpha_xy, alpha_yx = sympy.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x, epsilon_y, epsilon_xy = sympy.symbols('epsilon_x epsilon_y epsilon_xy')
epsilon = sympy.symbols('epsilon')
alpha_x = epsilon + 1
alpha_y = epsilon + 1

C00, C01, C10, C11 = sympy.symbols('C00 C01 C10 C11')
C_CM = sympy.Matrix([[C00, C01],
                     [C10, C11]])
I_C_CM = sympy.trace(C_CM) 
II_C_CM = (sympy.trace(C_CM)**2 - sympy.trace(C_CM**2))/2 
III_C_CM = sympy.det(C_CM) 
J_CM = sympy.sqrt(III_C_CM)
p = sympy.symbols('p')

F = sympy.Matrix(
        [[alpha_x,    0  ],\
        [   0   , alpha_y]])
J = F.det()
C = F.T * F
E = (C - sympy.eye(2))/2

## Quasi-incrompressible solid

### Microscopic model

In [ ]:
qois_filename = "Fig9-qois.dat"
res_basename = "Fig9"

Es = 1
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

load_params = {}

pf = 0
load_params["pf_lst"] = [pf, pf]
load_params["sigma_bar_01_lst"] = [0., 0.]
load_params["sigma_bar_10_lst"] = [0., 0.]
load_params["U_bar_00_lst"] = [0, 0.5]
load_params["U_bar_11_lst"] = [0, 0.5]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_00 = eps_xx_lst
Sigma_ter_p_00 = []

for i in range (len(eps_p_00)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_00.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])




pf = 0.25
load_params["pf_lst"] = [pf, pf]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_025 = eps_xx_lst
Sigma_ter_p_025 = []

for i in range (len(eps_p_025)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_025.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])



pf = 0.5
load_params["pf_lst"] = [pf, pf]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_05 = eps_xx_lst
Sigma_ter_p_05 = []

for i in range (len(eps_p_05)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_05.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])




pf = 1
load_params["pf_lst"] = [pf, pf]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_1 = eps_xx_lst
Sigma_ter_p_1 = []

for i in range (len(eps_p_1)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_1.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])

### Macroscopic model

In [ ]:
homogenization_problem = dmech.HomogenizationProblem(
    dim=2,
    mesh=mesh,
    mat_params=mat_params["parameters"],
    vertices=vertices,
    vol=vol,
    bbox=bbox)

lmbda_tilde, mu_tilde = homogenization_problem.get_lambda_and_mu()
beta_tilde = mu_tilde/2
alpha_tilde = lmbda_tilde/4


W_skel = beta_tilde * (I_C_CM - 2 - 2 * sympy.ln(J_CM)) + alpha_tilde * (J_CM**2 - 1 - 2 * sympy.ln(J_CM)) 
Sigma_CM = 2*sympy.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

Sigma = Sigma_CM.subs(list(zip(C_CM, C))).doit().as_explicit() 
sigma = F * Sigma * F.T / J 

Sigma_ter_macro = 2*sympy.diff(W_skel, C_CM)      
Sigma_ter_macro = Sigma_ter_macro.subs(list(zip(C_CM, C))).doit().as_explicit() 


lam_lin = sympy.lambdify(epsilon, Sigma_ter_macro[0,0], modules=['numpy'])

epsilon_ter_vals = numpy.linspace(0, 1, 100)
sigma_ter_vals = lam_lin(epsilon_ter_vals)

In [ ]:
plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\Sigma_{T}~(kPa)$', fontsize=16)
plt.plot(epsilon_ter_vals, sigma_ter_vals, '#99000D')
plt.plot(eps_p_00, Sigma_ter_p_00, '#084594')
plt.plot(eps_p_025, Sigma_ter_p_025, '#2171B5')
plt.plot(eps_p_05, Sigma_ter_p_05, '#4292C6')
plt.plot(eps_p_1, Sigma_ter_p_1, '#6BAED6')

plt.legend(['Macroscopic model 'r'$\bar{\Sigma}_{T}$'', any'r'$~\bar{p}_f$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.00~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.25~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.5~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=1.0~(kPa)$'])
plt.xlim(0, 0.5)
plt.ylim(0, 0.12)
# plt.savefig('Stress_ter_strain_pressure_incompressible.pdf',bbox_inches='tight')
plt.show()

## Compressible solid

### Microscopic model

In [ ]:
qois_filename = "Fig9-qois.dat"
res_basename = "Fig9"

Es = 1
nus = 0.2
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

load_params = {}

pf = 0
load_params["pf_lst"] = [pf, pf]
load_params["sigma_bar_01_lst"] = [0., 0.]
load_params["sigma_bar_10_lst"] = [0., 0.]

load_params["U_bar_11_lst"] = [0, 0.5]
load_params["U_bar_00_lst"] = [0, 0.5]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_00 = eps_xx_lst
Sigma_ter_p_00 = []

for i in range (len(eps_p_00)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_00.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])




pf = 0.25
load_params["pf_lst"] = [pf, pf]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_025 = eps_xx_lst
Sigma_ter_p_025 = []

for i in range (len(eps_p_025)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_025.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])



pf = 0.5
load_params["pf_lst"] = [pf, pf]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_05 = eps_xx_lst
Sigma_ter_p_05 = []

for i in range (len(eps_p_05)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_05.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])




pf = 1
load_params["pf_lst"] = [pf, pf]


dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":2, "Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_xx_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
eps_yy_lst = qois_vals[:, qois_name_list.index("U_bar_YY") - 1]
sigma_xx_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
sigma_yy_lst = qois_vals[:, qois_name_list.index("sigma_bar_YY") - 1]
p_lst = qois_vals[:, qois_name_list.index("p_f") - 1]

eps_p_1 = eps_xx_lst
Sigma_ter_p_1 = []

for i in range (len(eps_p_1)):
    U_bar = numpy.diag([eps_xx_lst[i], eps_yy_lst[i]])
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar

    sigma = numpy.diag([sigma_xx_lst[i], sigma_yy_lst[i]])
    Sigma = J_bar * numpy.linalg.inv(F_bar)* sigma * numpy.linalg.inv(F_bar.T)
    Sigma_ter_p_1.append((Sigma + p_lst[i]*J_bar*numpy.linalg.inv(C_bar))[0,0])

In [ ]:
plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\Sigma_{T}~(kPa)$', fontsize=16)
plt.plot(eps_p_00, Sigma_ter_p_00, '#084594')
plt.plot(eps_p_025, Sigma_ter_p_025, '#2171B5')
plt.plot(eps_p_05, Sigma_ter_p_05, '#4292C6')
plt.plot(eps_p_1, Sigma_ter_p_1, '#6BAED6')

plt.legend(['Macroscopic model 'r'$\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.00~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.25~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=0.5~(kPa)$', r'Microscopic model $\tilde{\Sigma}_{T}$, $\tilde{p}_f=1.0~(kPa)$'])
plt.xlim(0, 0.5)
plt.ylim(0, 0.12)
# plt.savefig('Stress_ter_strain_pressure_compressible.pdf',bbox_inches='tight')
plt.show()